# Data Preprocessing for Metal Nanoparticle Surface Atoms Clustering

This jupyter notebook is used to preprocess the collected tabular data from metal nanoparticle data sets.

## Index:
- [Requirements](#requirements)
- [Import Libraries](#import-libraries)
- [Load Data](#load-data)
- [Data Preprocessing](#data-preprocessing)
    - [Description & Massage](#desc-massage)
    - [Data Scaling](#data-scaling)
    - [Feature Selection](#feat-select)
    - [Data Visualisation](#data-vis)
- [Test Cases](#test-cases)
- [Variables Storage](#var-store)

<a id='requirements'></a>
## Requirements

The following files should be available in a specified directory (./data/ here):
* "dfAlls.pickle"

<a id='import-libraries'></a>
## Import Libraries

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
import sys
import warnings

import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rcParams
import numpy as np
# import modin.pandas as modpd
import pandas as pd
import pickle
import seaborn as sns
import sklearn
from statsmodels.graphics.gofplots import qqplot

# Dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Machine Learning Miscellaneous
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler, StandardScaler, PowerTransformer, QuantileTransformer
from scipy.stats import probplot, shapiro, normaltest, anderson, skew, kurtosis

In [146]:
# Set up notebook environment
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Settings for figures
sns.set_theme(context='paper', style='ticks', palette='colorblind', font='sans-serif', font_scale=1, color_codes=True, rc=None)  # Options are: {paper, notebook, talk, poster}
FIG_SIZE, DPI, fontSize, labelSize = (3.5, 2.5), None, 'medium', 'small'
legendSize, lineWidth, markerSize = 'x-small', 1, 3
SMALL_SIZE, MEDIUM_SIZE, LARGE_SIZE, TITLE_SIZE = 8, 10, 12, 14
plt.rc('font', size=LARGE_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=TITLE_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=LARGE_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=LARGE_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=LARGE_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc('figure', titlesize=TITLE_SIZE)  # fontsize of the figure title
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['figure.figsize'] = [8, 5]
# rcParams['figure.autolayout'] = True
# rcParams['font.style'] = 'normal'
# rcParams['xtick.labelsize'] = 10
# rcParams['ytick.labelsize'] = 10

warnings.filterwarnings("ignore")
%pprint

Pretty printing has been turned OFF


In [7]:
# Print version for reproducibility
print(f"matplotlib version {mpl.__version__}")
print(f"numpy version {np.__version__}")
print(f"pandas version {pd.__version__}")
print(f"seaborn version {sns.__version__}")
print(f"sklearn version {sklearn.__version__}")

matplotlib version 3.5.1
numpy version 1.21.5
pandas version 1.3.5
seaborn version 0.11.2
sklearn version 1.0.2


<a id='load-data'></a>
## Load Data

In [9]:
%store -r testCases
RANDOM_SEED = 42
NUM_JOBS = 8
CORR_METHOD = 'pearson'
varThreshs, corrThreshs = [0.0], [0.9]
reinitialise = False
if reinitialise:
    dfScaledNoLVHCs = np.array([[[None] * len(testCases)] * len(corrThreshs)] * len(varThreshs))
    dfNoLVHCs = np.array([[[None] * len(testCases)] * len(corrThreshs)] * len(varThreshs))
    corrMatNoLVHCs = np.array([[[None] * len(testCases)] * len(corrThreshs)] * len(varThreshs)) 
    %store -d dfScaledNoLVHCs dfNoLVHCs corrMatNoLVHCs
    %store dfScaledNoLVHCs dfNoLVHCs corrMatNoLVHCs

with open('data/dfAlls.pickle', 'rb') as f: 
    dfAlls = pickle.load(f)
# for (i, testCase) in enumerate(testCases):
#     print("{0} {1}".format(i, testCase))
#     display(dfAlls[i].sample(5))

<a id='data-preprocessing'></a>
## Data Preprocessing

<a id='desc-massage'></a>
### Description & Massage

In [10]:
def dfDesc(df, interactive=True):
    """
    Describe input DataFrame using df.describe() and df.info()
    input:
        df = input DataFrame
        interactive = Boolean indicator to decide usage of function (display/print) for DataFrame inspection
    """
    print("\nOriginal DataFrame:")
    display(df.sample(5)) if interactive else print(df.sample(5))
    print("\nDataFrame description:")
    display(df.describe()) if interactive else print(df.describe())
    print("\nDataFrame information:")
    df.info()

In [13]:
def dfMassage(df, verbose=False, interactive=True):
    """
    Handle missing values, drop duplicates in input DataFrame, one-hot encode certain features, and turn columns datatype into numeric
    input:
        df = input DataFrame
        verbose = Boolean indicator for output printing
        interactive = Boolean indicator to decide usage of function (display/print) for DataFrame inspection
    output:
        dfNew = processed DataFrame
    """
    # Handle missing values
    if df.isna().any().any():
        print("Missing entries exist!")
        missingNum = df.isna().sum()
        missingNum[missingNum > 0].sort_values(ascending=False)
        print(f"Missing rows in each column/feature:\n{missingNum}")
        # df.dropna(axis=0, how='any', thresh=None, subset=['pg', 'csm', 'molRMS'], inplace=True)

    # Drop duplicates
    if df.duplicated().any():
        print("Duplicate entries exists!")
        print(f"Number of rows before dropping duplicates: {len(df)}")
        df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False);
        print(f"Number of rows after dropping duplicates: {len(dfNew)}")
        
    # One-hot encode features
    dummyEle = pd.get_dummies(df['ele'], prefix='ele')
    dfNew = pd.merge(left=df, right=dummyEle, left_index=True, right_index=True)
    
    # To numeric
    dfNew = dfNew.astype({'xNorm': float, 'yNorm': float, 'zNorm': float, 
                          'Ixx': float, 'Iyy': float, 'Izz': float, 
                          'degenDeg': float})
    
    print("\nMissing values handled, duplicates dropped, one-hot encoded categorical features, feature columns numericised:")
    if verbose: display(dfNew.sample(5)) if interactive else print(dfNew.sample(5))
    return dfNew

In [14]:
def dfDropUnusedCols(df):
    """
    Drop unused feature columns (could vary depending on datasets e.g. mono/multi-metallic)
    input:
        df = input DataFrame
    output:
        dfNew = processed DataFrame
    """
    # 'x', 'y', 'z' are redundant (included normalised columns)
    # 'pg' could only be one-hot encoded when all possibilities are recorded
    # 'csm', 'molRMS' undefined when 'pg' is missing
    # inclusion of "E", "C8", etc. makes clustering difficult (binary features become dominant)
    pgIdx = list(df.columns).index('pg')
    surfIdx = list(df.columns).index('surf')
    cols2drop = ['x', 'y', 'z', 'ele'] + list(df.columns)[pgIdx:surfIdx]  # Could simply leave 'ele' to removal of features with low variance
    dfNew = df.drop(labels=cols2drop, axis=1, index=None, columns=None, level=None, inplace=False, errors="raise")
    dfNew.rename(columns={'xNorm': 'x', 'yNorm': 'y', 'zNorm': 'z'}, inplace=True)
    return dfNew

<a id='data-scaling'></a>
### Data-Scaling

In [25]:
def dfScale(df, catCols=None, scaler="minMax"):
    """
    Scale input feature DataFrame using various sklearn scalers
    input:
        df = DataFrame with columns being features
        catCols = list of categorical features, not to be scaled
        scaler = type of sklearn scaler to use
    output:
        Xscaled = scaled DataFrames
    """
    X4keep = df[catCols] if catCols else None
    X4scale = df.drop(X4keep.columns, axis=1, inplace=False) if catCols else df
    if scaler == "minMax":
        scaler = MinMaxScaler(feature_range=(0, 1), copy=True).fit(X4scale)
    elif scaler == "stand":
        scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X4stand)
    elif scaler == "robust":
        scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True).fit(X4scale)
    else:
        raise("\nScaler specified unknown!")
    arrXscaled = scaler.transform(X4scale)
    Xscaled = pd.concat(objs=[pd.DataFrame(arrXscaled, index=df.index, columns=X4scale.columns), X4keep], axis=1)
    Xscaled.index.name = "Index"
    return Xscaled

<a id='feat-select'></a>
### Feature Selection

In [26]:
def getLowVarCols(df, skipCols=None, varThresh=0.0, autoRemove=False, verbose=False):
    """
    Wrapper for sklearn VarianceThreshold.
    input:
        df = DataFrame with columns being features
        skipCols = columns to be skipped
        varThresh = low variance threshold for features to be detected
        autoRemove = Boolean indicator for automatic removal of low variance columns
        verbose = Boolean indicator for output printing
    output:
        df = DataFrame with low variance features removed
        lowVarCols = list of low variance features
    """
    print(f"\n  Finding features with low-variance, threshold: {varThresh}")
    try:
        allCols = df.columns
        if skipCols:
            remainCols = allCols.drop(skipCols)
            maxIdx = len(remainCols) - 1
            skippedIdx = [allCols.get_loc(col) for col in skipCols]

            # Adjust insert location by the number of columns removed (for non-zero insertion locations) to keep relative locations intact
            for idx, item in enumerate(skippedIdx):
                if item > maxIdx:
                    diff = item - maxIdx
                    skippedIdx[idx] -= diff
                if item == maxIdx:
                    diff = item - len(skipCols)
                    skippedIdx[idx] -= diff
                if idx == 0:
                    skippedIdx[idx] = item
            skippedVals = df.iloc[:, skippedIdx].values
        else:
            remainCols = allCols

        X = df.loc[:, remainCols].values
        vt = VarianceThreshold(threshold=varThresh)
        vt.fit(X)
        keepColsIdxs = vt.get_support(indices=True)
        keepCols = [remainCols[idx] for idx, _ in enumerate(remainCols) if idx in keepColsIdxs]
        lowVarCols = list(np.setdiff1d(remainCols, keepCols))
        if verbose: print("    Found {0} low-variance columns.".format(len(lowVarCols)))

        if autoRemove:
            if verbose: print("    Removing low-variance features...")
            X_removed = vt.transform(X)
            if verbose: print("    Reassembling the dataframe (with low-variance features removed)...")
            df = pd.DataFrame(data=X_removed, columns=keepCols)
            if skipCols:
                for (i, index) in enumerate(skippedIdx): df.insert(loc=index, column=skipCols[i], value=skippedVals[:, i])
            if verbose: print("    Succesfully removed low-variance columns: {0}.".format(lowVarCols))
        else:
            if verbose: print("    No changes have been made to the dataframe.")
    except Exception as e:
        print(e)
        print("    Could not remove low-variance features. Something went wrong.")
    return df, lowVarCols

In [27]:
def getHighCorCols(df, corrThresh=0.95, method="pearson", verbose=False):
    """
    Compute correlation matrix using pandas
    input: 
        df = input DataFrame with columns being features
        corrThresh = threshold to identify highly correlated features
        method = method to compute DataFrame correlation
        verbose = Boolean indicator for output printing
    output: 
        corrMat = correlation matrix of all features in input DataFrame
        highCorCols = tuples of highly correlated features
    """ 
    print(f"\n  Finding features highly-correlated with each other, threshold: {corrThresh}")
    corrMat = df.corr(method=method, min_periods=1)
    corrMatUpper = corrMat.where(np.triu(np.ones(corrMat.shape), k=1).astype(np.bool))  # Select upper triangle of correlation matrix
    highCorCols = [(row, col) for col in corrMatUpper.columns for row in corrMatUpper.index if corrMatUpper.loc[row, col] > corrThresh]
    if verbose: print(f"    Highly correlated columns: {highCorCols}")
    return corrMat, highCorCols

In [148]:
def plotCorrMat(corrMat, figSize=(8, 8), figName=None):
    """
    Wrapper for sklearn VarianceThreshold.
    input:
        corrMat = correlation matrix of all features in input DataFrame
        figSize = size of figure
        figName = path to save figure
    """
    sns.set(font_scale=1.1)
    # cmap = sns.diverging_palette(h_neg=210, h_pos=350, s=90, l=50, as_cmap=True)
    g = sns.clustermap(data=corrMat.abs().mul(100).astype(float), cmap='Blues', metric='correlation', figsize=figSize);
    plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0);
    # x0, _y0, _w, _h = g.cbar_pos
    # print(g.cbar_pos)
    g.ax_cbar.set_position([0.04, 0.82, 0.05, 0.14]);
    if figName: 
        plt.savefig(figName, dpi=DPI, bbox_inches='tight');
    plt.close();

In [29]:
def autoDrop(highCorCols, verbose=False):
    """
    Automate the selection of highly-correlated features to be dropped. 
    Rank given to each feature is based on its degree of utility/ease of interpretation if it turns out to be correlated with target labels
    'x', 'y', 'z' not very useful even if found to be important
    'avg' values of bond geometries are more useful than 'num' values
    'max', 'min' of bond geometries are hard to control experimentally
    'angParam', 'centParam' are easier to interpret than 'entroParam'
    'disorder' parameters are easier to interpret than 'q' Steinhardt's parameters
    averaged 'q' parameters are more robust to thermal fluctuations, hence more useful than pure 'q' parameters
    q6 > q4 > q2 == q8 == q10 == q12 in usefulness based on literature, thus 'disorder' parameters follow same sequence
    input:
        highCorCols = list of tuples of highly-correlated features
        verbose = Boolean indicator for output printing
    output: 
        cols2drop = list of features to be dropped
    """
    utilityRanking = {'x': 10, 'y': 10, 'z': 10, 'rad': 0, 
                     'blavg': 2, 'blmax': 8, 'blmin': 8, 'blnum': 3, 
                     'ba1avg': 2, 'ba1max': 8, 'ba1min': 8, 'ba1num': 3,
                     'ba2avg': 2, 'ba2max': 8, 'ba2min': 8, 'ba2num': 3,
                     'btposavg': 2, 'btposmax': 8, 'btposmin': 8, 'btposnum': 3, 
                     'btnegavg': 2, 'btnegmax': 8, 'btnegmin': 8, 'btnegnum': 3, 
                     'cn': 1, 'gcn': 0, 'scn': 3, 'sgcn': 3, 'q6q6': 2, 
                     'Ixx': 5, 'Iyy': 5, 'Izz': 5, 'degenDeg': 6, 
                     'angParam': 4, 'centParam': 4, 'entroParam': 5, 'entroAvgParam': 5.5, 
                     'chi1': 6, 'chi2': 6, 'chi3': 6, 'chi4': 6, 'chi5': 6, 'chi6': 6, 'chi7': 6, 'chi8': 6, 'chi9': 6, 
                     'q2': 5.7, 'q4': 5.6, 'q6': 5.5, 'q8': 5.7, 'q10': 5.7, 'q12': 5.7, 
                     'q2avg': 5.2, 'q4avg': 5.1, 'q6avg': 5, 'q8avg': 5.2, 'q10avg': 5.2, 'q12avg': 5.2, 
                     'disord2': 4.7, 'disord4': 4.6, 'disord6': 4.5, 'disord8': 4.7, 'disord10': 4.7, 'disord12': 4.7, 
                     'disordAvg2': 4.2, 'disordAvg4': 4.1, 'disordAvg6': 4, 'disordAvg8': 4.2, 'disordAvg10': 4.2, 'disordAvg12': 4.2
                     }  # Lower score = Higher rank

    # occurCount = Counter(list(sum(highCorCols, ())))
    print("\n    Sorting all highly-correlated feature pairs based on their minimum and total utility rankings.")
    highCorColsProps = []
    for (col1, col2) in highCorCols:
        rank1, rank2 = utilityRanking[col1], utilityRanking[col2]
        highCorColsProps.append((min(rank1, rank2), rank1 + rank2))
    sortedIdx = sorted(range(len(highCorColsProps)), key=lambda i: highCorColsProps[i])
    highCorCols, highCorColsProps = [highCorCols[i] for i in sortedIdx], [highCorColsProps[i] for i in sortedIdx]
    
    print("\n    Removing one of each highly-correlated feature pairs.")
    cols2drop = []
    for (i, (col1, col2)) in enumerate(highCorCols):
        if verbose: 
            print(f"      Feature pairs: {col1} {col2}")
        if col1 in cols2drop or col2 in cols2drop:
            if verbose: 
                print("        One of the features is dropped, skip this pair.\n")
            continue
        elif utilityRanking[col1] > utilityRanking[col2]:
            if verbose: 
                print(f"        {col1} has lower utility score compared to {col2}")
            cols2drop.append(col1)
        else:
            if verbose: 
                print(f"        {col2} has lower utility score compared to {col1}")
            cols2drop.append(col2)
    if verbose: 
        print(f"    Feature columns to drop: {cols2drop}")
    return cols2drop

In [30]:
def varCorrDropCols(X, varThresh=0.00, corrThresh=0.90, figName=None, verbose=False):
    """
    Remove features with low variance and one of the highly-correlated feature pairs
    input:
        X = input scaled DataFrame with each column being feature
        varThresh = threshold below which feature is removed
        corrThresh = threshold above which one of each pair of correlated features is removed
        verbose = Boolean indicator for output printing
    output:
        XNoLVHC = DataFrame with the undesired features removed
        corrMatNoLVHC = computed correlated matrix
    """
    # Remove columns with low variance
    XNoLV, lowVarCols = getLowVarCols(df=X, skipCols=None, varThresh=varThresh, autoRemove=True)
    
    # Remove one of the feature columns that are highly correlated with each other
    corrMatNoLV, highCorCols1 = getHighCorCols(df=XNoLV, corrThresh=corrThresh, method=CORR_METHOD)
    
    cols2drop = autoDrop(highCorCols1, verbose=verbose)
    XNoLVHC = XNoLV.drop(labels=cols2drop, axis=1, index=None, columns=None, level=None, inplace=False, errors="raise")
    corrMatNoLVHC, highCorCols2 = getHighCorCols(df=XNoLVHC, corrThresh=corrThresh, method=CORR_METHOD)
    plotCorrMat(corrMat=corrMatNoLVHC, figSize=(8, 8), figName=figName)
    return XNoLVHC, corrMatNoLVHC

<a id='data-vis'></a>
### Data Visualisation

##### t-SNE
- perplexity = related to number of nearest neighbours used in other manifold learning algorithms, larger datasets need larger value, range = (5-50), different values give significantly different results
- early_exaggeration = controls how tight natural clusters in original space are in embedded space, proportional to space between clusters in embedded space
- learning_rate = range = \[10.0, 1000.0\], too high = ball with equidistant points, too low = most points look compressed in a dense cloud with some outliers

In [160]:
def vis2D(caseID, varThreshIdx=0, corrThreshIdx=1, colourLabel='surf',
          perplexity=30.0, early_exaggeration=12, metric='euclidean'):
    """
    Visualise the atoms in 2D manifold using PCA, t-SNE and UMAP
    input:
        caseID = ID index of nanoparticle of interest
        varThresh = threshold below which feature is removed
        corrThresh = threshold above which one of each pair of correlated features is removed
        * Refer to sklearn and umap packages for other input variables
    """
    %store -r dfScaledNoLVHCs
    df = dfScaledNoLVHCs[varThreshIdx][corrThreshIdx][caseID]
    X, y = df.iloc[:, :-1], df[colourLabel]
    tsne = TSNE(n_components=2, 
                perplexity=perplexity, early_exaggeration=early_exaggeration, learning_rate='auto', 
                n_iter=1000, n_iter_without_progress=300, min_grad_norm=1e-7, metric=metric, 
                random_state=RANDOM_SEED, init='pca', verbose=0, 
                method='barnes_hut', angle=0.5, n_jobs=NUM_JOBS)
    redTSNE = tsne.fit_transform(X)

    fig, ax = plt.subplots(figsize=(3, 3))
    colours = ['#FF5252' if isSurf else '#303F9F' for isSurf in y] if colourLabel == 'surf' else y
    plt.scatter(redTSNE[:, 0], redTSNE[:, 1], s=3, c=colours, cmap=plt.get_cmap('tab10', len(y.unique())))
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.grid(linestyle='dotted')
    plt.savefig(f"figures/dimRedVis/{testCases[caseID]}DRvis_V{int(varThreshs[varThreshIdx]*100)}C{int(corrThreshs[corrThreshIdx]*100)}{colourLabel}.png",
                dpi=DPI, bbox_inches='tight')
    plt.close()

In [170]:
def expVarPCA(caseID, varThreshIdx=0, corrThreshIdx=1, verbose=False):
    """
    Calculate and plot the variance explained by PCA components
    input:
        caseID = ID index of nanoparticle of interest
        varThresh = threshold below which feature is removed
        corrThresh = threshold above which one of each pair of correlated features is removed
        * Refer to sklearn package for other input variables
    """
    %store -r dfScaledNoLVHCs
    pca = PCA(n_components=None, copy=True, whiten=False, svd_solver='auto', tol=0.0, 
              iterated_power='auto', random_state=RANDOM_SEED)
    redPCA = pca.fit_transform(dfScaledNoLVHCs[varThreshIdx][corrThreshIdx][caseID].iloc[:, :-1])
    if verbose:
        # print("PCA components:\n", pca.components_)
        # print("Explained Variance:\n", pca.explained_variance_)
        print("\nExplained Variance Ratio:\n", pca.explained_variance_ratio_)
    plt.figure(figsize=(4, 3.5))
    plt.plot(np.cumsum(pca.explained_variance_ratio_), marker=None, color='k')
    plt.xlabel('Number of principal components')
    plt.ylabel('Cumulative explained variance')
    plt.grid(linestyle='dotted')
    plt.savefig(f"figures/PCAexpVar/{testCases[caseID]}PCA_V{int(varThreshs[varThreshIdx]*100)}C{int(corrThreshs[corrThreshIdx]*100)}.png",
                dpi=DPI, bbox_inches='tight')
    plt.close()

In [171]:
def dimRedVis(caseID, varThreshIdxs=[0], corrThreshIdxs=[1], colourLabel='surf', verbose=False):
    """
    Visualise data with reduced dimensions. 
    input:
        caseID = ID index of nanoparticle of interest
        varThresh = threshold below which feature is removed
        corrThresh = threshold above which one of each pair of correlated features is removed
        * Refer to sklearn package for other input variables
    output:
    """
    %store -r dfScaledNoLVHCs dfNoLVHCs
    for varThreshIdx in varThreshIdxs:
        for corrThreshIdx in corrThreshIdxs:
            vis2D(caseID=caseID, varThreshIdx=varThreshIdx, corrThreshIdx=corrThreshIdx, colourLabel=colourLabel)
            expVarPCA(caseID=caseID, varThreshIdx=varThreshIdx, corrThreshIdx=corrThreshIdx, verbose=verbose)
            if verbose:
                print("\nScaled DataFrame, varThresh: {0}, corrThresh {1}:".format(varThreshs[varThreshIdx], corrThreshs[corrThreshIdx]))
                # display(dfScaledNoLVHCs[varThreshIdx][corrThreshIdx][caseID].sample(5))
                print("\nUnscaled DataFrame, varThresh: {0}, corrThresh {1}:".format(varThreshs[varThreshIdx], corrThreshs[corrThreshIdx]))
                # display(dfNoLVHCs[varThreshIdx][corrThreshIdx][caseID].sample(5))

In [172]:
def dataPreProc(caseID, verbose=False, interactive=True):
    """
    Run the data preprocessing pipeline for a nanoparticle
    input:
        caseID = ID index of nanoparticle of interest
        verbose = Boolean indicator of whether text outputs should be elaborated 
        interactive = Boolean indicator of whether DataFrames should be displayed (to be investigated in Notebook) or printed (to be stored in .txt file)
    output:
        dfNoLVHC = unscaled DataFrame without highly-correlated and low variance features
        dfScaledNoLVHC = scaled DataFrame without highly-correlated and low variance features
        corrMatNoLVHC = computed correlation matrix
    """
    print("\nTest case: {0}".format(testCases[caseID]))
    # dfDesc(df=dfAlls[caseID], interactive=interactive)
    dfCleaned = dfMassage(df=dfAlls[caseID], verbose=verbose)
    dfDropped = dfDropUnusedCols(df=dfCleaned)
    dfScaled = dfScale(df=dfDropped, catCols=['surf'], scaler='minMax')
    Xscaled, y = dfScaled.iloc[:, :-1], dfScaled.loc[:, 'surf']
    
    %store -r dfScaledNoLVHCs dfNoLVHCs corrMatNoLVHCs
    for (i, varThresh) in enumerate(varThreshs):
        for (j, corrThresh) in enumerate(corrThreshs):
            print(f"\nRemove features with varThresh: {varThresh}, corrThresh: {corrThresh}")
            varCorrDropColsFigName = f"figures/corrMat/{testCases[caseID]}cmatV{int(varThresh*100)}C{int(corrThresh*100)}.png"
            XScaledNoLVHC, corrMatNoLVHC = varCorrDropCols(X=Xscaled, varThresh=varThresh, corrThresh=corrThresh, figName=varCorrDropColsFigName, verbose=verbose)
            dfScaledNoLVHC = pd.concat(objs=[XScaledNoLVHC, y], axis=1, join="inner", ignore_index=False)
            dfNoLVHC = dfDropped[dfScaledNoLVHC.columns]
            dfScaledNoLVHCs[i][j][caseID] = dfScaledNoLVHC
            dfNoLVHCs[i][j][caseID] = dfNoLVHC
            corrMatNoLVHCs[i][j][caseID] = corrMatNoLVHC
    %store dfScaledNoLVHCs dfNoLVHCs corrMatNoLVHCs

# dataPreProc(0, verbose=False, interactive=False)

<a id='test-cases'></a>
## Test Cases

In [190]:
# i=0

In [299]:
%%capture dataProcPd --no-stderr
if i < len(testCases): 
    dataPreProc(caseID=i, verbose=True)

In [300]:
%%capture dimRedVisPd --no-stderr
if i < len(testCases): 
    dimRedVis(caseID=i, varThreshIdxs=[0], corrThreshIdxs=[0], colourLabel='surf', verbose=True)

In [301]:
print(i) if i < len(testCases) else print("No more test cases!")
with open(f"dataPreprocOutputs/dataProc{i}Pd.txt", 'w') as f: 
    f.write(dataProcPd.stdout)
with open(f"dataPreprocOutputs/dimRedVis{i}Pd.txt", 'w') as f: 
    f.write(dimRedVisPd.stdout)
i += 1

No more test cases!


<a id='var-store'></a>
## Variables Storage

In [302]:
with open('data/dfScaledNoLVHCs.pickle', 'wb') as f: 
    pickle.dump(dfScaledNoLVHCs, f)
with open('data/dfNoLVHCs.pickle', 'wb') as f: 
    pickle.dump(dfNoLVHCs, f)
with open('data/corrMatNoLVHCs.pickle', 'wb') as f: 
    pickle.dump(corrMatNoLVHCs, f)

In [304]:
# Check the output
with open("data/dfScaledNoLVHCs.pickle", 'rb') as f: 
    dfScaledNoLVHCs = pickle.load(f)
for (i, dfScaledNoLVHC) in enumerate(dfScaledNoLVHCs[0][0]):  # varThresh = 0.0, corrThresh = 0.9
    display(dfScaledNoLVHC.sample(5))

,x,y,z,ba1max,ba2avg,btposavg,gcn,scn,rad,angParam,chi1,q2avg,q4avg,q6avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord10,disordAvg10,Ixx,Izz,surf
658,0.416667,0.000000,0.833333,1.0,0.134615,0.861314,0.128205,0.833333,0.922688,0.947337,0.166667,0.624169,3.886436e-01,3.141947e-01,0.631017,0.668046,0.684299,0.620094,0.469537,0.376298,0.868289,0.754447,0.514120,0.571429,1
825,0.916667,0.583333,0.916667,1.0,0.692308,0.408759,0.320513,1.000000,0.901734,0.947312,0.500000,0.561141,1.610393e-01,3.132562e-01,0.349849,0.426047,0.334704,0.256124,0.539059,0.322824,0.488109,0.461691,0.614048,0.727273,1
101,0.583333,0.166667,0.166667,1.0,0.557692,0.211679,0.897436,0.000000,0.697254,0.947370,0.666667,0.329670,5.277001e-02,1.095470e-01,0.753067,0.684929,0.207978,0.269500,0.000033,0.329685,0.580290,0.692956,0.934468,0.012987,0
795,0.666667,0.250000,1.000000,1.0,0.000000,1.000000,0.141026,0.833333,0.881503,0.947370,0.166667,0.762825,5.937587e-01,4.601113e-01,0.617072,0.452006,0.183304,0.230734,0.789440,0.782752,0.602240,0.594745,0.504707,0.610390,1
504,0.333333,0.583333,0.666667,1.0,0.269231,0.656934,1.000000,0.000000,0.294075,0.947370,0.666667,0.000052,5.476407e-08,1.624765e-07,0.744267,0.778581,0.207978,0.153542,0.000033,0.000018,0.580290,0.567990,0.934468,0.012987,0


,x,y,z,blavg,blmax,ba1avg,ba1max,ba1min,ba2avg,btposavg,btnegavg,gcn,rad,q6q6,angParam,chi1,chi3,q2,q4,q6,q10,q2avg,q6avg,disordAvg2,disord4,disordAvg4,disordAvg6,disordAvg8,disord12,centParam,Ixx,Izz,surf
569,0.000000,0.500000,0.809023,1.000000,1.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.999957,0.000000,0.0,0.511067,0.509461,0.803766,0.931183,0.203807,0.133810,0.376960,0.349169,0.320351,0.889437,0.495082,0.396337,0.999916,0.006894,0.058537,1
428,0.448496,0.789349,0.833333,0.606557,1.0,0.960699,1.0,0.0,0.701493,0.039604,0.960396,0.844156,0.755841,0.666667,0.203534,0.500000,0.6,0.115345,0.559565,0.220129,0.000100,0.417486,0.396252,0.364916,0.426740,0.231150,0.552847,0.887965,0.443058,0.207261,0.998030,0.829268,0
415,0.396992,0.178822,0.250000,0.606557,1.0,0.960699,1.0,0.0,0.656716,0.198020,0.801980,0.844156,0.714369,1.000000,0.203534,0.500000,0.6,0.115340,0.559529,0.220119,0.000101,0.476566,0.616341,0.322053,0.450943,0.150672,0.611096,0.797386,0.442646,0.207266,0.979317,0.619512,0
693,0.757520,0.031829,0.583333,0.901639,1.0,0.672489,1.0,0.0,0.447761,0.386139,0.613861,0.220779,0.919977,0.333333,0.769360,0.166667,0.2,0.999778,0.597227,0.457734,0.443994,0.622542,0.365920,0.326956,0.282076,0.353729,0.863651,0.784038,0.856675,0.402447,0.546619,0.068293,1
380,0.210651,0.833333,0.551504,0.606557,1.0,0.960699,1.0,0.0,0.701493,0.039604,0.960396,0.844156,0.755841,0.666667,0.350823,0.333333,0.6,0.115352,0.559535,0.220128,0.000090,0.417496,0.396247,0.624446,0.637171,0.372270,0.476265,0.535152,0.412752,0.207274,0.882141,0.707317,0


,x,y,z,ba1avg,ba1max,ba2avg,btposavg,gcn,scn,rad,angParam,entroAvgParam,q4,q4avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord10,disordAvg10,disord12,Izz,surf
701,0.357143,0.285714,0.857143,0.802469,1.0,0.175439,0.707143,0.423077,1.0,0.809432,0.999672,0.552821,3.250711e-01,6.694294e-01,0.399241,0.533052,0.308105,0.337232,0.731644,0.656816,0.534607,0.530685,0.565827,0.701299,1
577,0.428571,0.142857,0.642857,1.000000,1.0,0.596491,0.228571,0.897436,0.0,0.718992,0.999672,0.182259,1.657348e-08,7.516610e-02,0.750277,0.624621,0.134236,0.145910,0.013857,0.216546,0.457575,0.453064,0.064431,0.961039,0
431,0.357143,0.714286,0.571429,1.000000,1.0,0.333333,0.550000,1.000000,0.0,0.490956,0.999738,0.000009,4.972321e-09,3.141712e-08,1.000000,0.999343,0.134205,0.051752,0.013841,0.000000,0.456854,0.413386,0.064401,0.961039,0
594,0.428571,0.642857,0.714286,1.000000,1.0,0.333333,0.550000,1.000000,0.0,0.490956,0.999738,0.000005,4.972321e-09,1.748062e-08,0.250877,0.429494,0.134298,0.051881,0.013890,0.000038,0.459017,0.416137,0.064491,0.961039,0
138,0.500000,0.428571,0.285714,1.000000,1.0,0.333333,0.600000,1.000000,0.0,0.415375,0.999672,0.000010,1.657348e-08,3.559846e-08,0.750317,0.662341,0.134236,0.051825,0.013857,0.000021,0.457575,0.414941,0.064431,0.012987,0


,x,y,z,ba1avg,ba1max,ba2avg,btposavg,btnegavg,gcn,scn,rad,angParam,entroParam,entroAvgParam,chi1,q4,q4avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord10,disordAvg10,Ixx,Izz,surf
420,0.166667,0.833333,0.416667,1.0,1.0,0.224138,0.341935,0.658065,0.987013,0.000000,0.610373,0.941110,0.000009,0.020545,1.000000,4.972120e-09,0.004052,0.255034,0.439974,0.090855,0.116883,0.003207,0.103263,0.256355,0.396046,1.000000,1.000000,0
938,0.500000,0.000000,0.916667,0.0,0.0,0.000000,1.000000,0.000000,0.051948,0.833333,0.875395,0.941138,0.999982,0.812991,0.000000,4.143433e-08,0.430459,0.481757,0.674130,0.000070,0.537905,0.000036,0.430006,0.050386,0.606116,0.373642,0.532468,1
177,0.750000,0.083333,0.083333,1.0,1.0,0.793103,0.174194,0.825806,0.519481,1.000000,0.858950,0.705833,0.173942,0.707859,0.833333,2.282651e-01,0.046524,0.377772,0.518587,0.179604,0.186822,0.235793,0.128392,0.382347,0.232392,0.996017,0.870130,1
971,0.583333,0.416667,0.916667,1.0,1.0,0.517241,0.348387,0.651613,0.831169,0.000000,0.539532,0.941124,0.000009,0.321030,1.000000,4.972120e-09,0.322855,0.255034,0.194868,0.090855,0.185796,0.003207,0.027132,0.256355,0.126494,0.934468,0.012987,0
237,0.083333,0.000000,0.333333,0.0,0.0,0.000000,1.000000,0.000000,0.051948,0.833333,0.908286,0.999803,0.999991,0.812999,0.000000,2.320138e-08,0.430414,0.820698,0.874147,0.000035,0.735160,0.000018,0.448741,0.049396,0.956082,0.254888,0.000000,1


,x,y,z,ba1avg,ba1max,ba2avg,btposavg,btnegavg,gcn,scn,rad,angParam,entroAvgParam,chi1,q6,q4avg,q6avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord10,Ixx,Izz,surf
992,0.333333,0.916667,1.000000,0.69802,1.0,0.365854,0.617778,0.382222,0.295455,1.0,0.798221,0.312389,0.695178,0.333333,3.252635e-02,7.029196e-01,0.662639,0.495139,0.584011,0.326461,0.532803,0.257958,0.424708,0.433127,0.648421,0.012987,1
657,0.416667,0.000000,0.666667,0.69802,1.0,0.085366,0.768889,0.231111,0.397727,0.8,0.610339,0.937447,0.470121,0.333333,3.241780e-02,9.997600e-01,0.118967,0.982759,0.950044,0.999941,0.999914,0.277576,0.237195,0.999992,0.692281,0.623377,1
778,0.083333,0.166667,0.833333,1.00000,1.0,0.439024,0.266667,0.733333,0.852273,0.0,0.740411,0.312399,0.260377,1.000000,4.034982e-08,3.228352e-01,0.036541,0.364936,0.278530,0.328738,0.181958,0.231401,0.174777,0.594771,0.936491,0.012987,0
309,0.250000,0.666667,0.333333,1.00000,1.0,0.195122,0.600000,0.400000,1.000000,0.0,0.352974,0.937501,0.000020,0.500000,1.284606e-07,7.257147e-08,0.000001,0.465178,0.328843,0.328701,0.157962,0.231388,0.143802,0.594094,1.000000,1.000000,0
226,0.750000,1.000000,0.166667,0.69802,1.0,0.085366,0.688889,0.311111,0.397727,0.8,0.769316,0.937501,0.470128,0.166667,3.249925e-02,9.999348e-01,0.119359,0.982756,0.946933,0.999977,0.999975,0.277614,0.237301,0.999997,0.692982,0.649351,1


...

,x,y,z,blavg,blmax,blmin,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,angParam,entroAvgParam,chi1,chi2,chi3,chi4,chi7,q4,q6,q8,q10,q12,q2avg,q6avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg6,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Izz,surf
99,0.500268,0.361839,0.191135,0.568323,0.725038,0.368812,0.828452,0.951613,0.352941,0.282609,0.500000,0.0,0.574074,0.344037,1.0,0.0,1.0,0.0,0.483146,0.75,0.435484,0.489095,0.474413,0.333333,0.5,0.0,0.0,0.0,0.508145,0.442993,0.618267,0.529609,0.290233,0.605781,0.722005,0.316995,0.549824,0.731229,0.763876,0.223179,0.163415,0.411686,0.638007,0.367826,0.240226,0.368586,0.732710,0.206478,1
366,0.396610,0.350976,0.437633,0.714286,0.470046,0.393564,0.974895,0.935484,0.588235,0.347826,0.454545,0.0,0.467593,0.357798,1.0,0.0,1.0,0.0,0.988764,0.00,0.264516,0.706283,0.019931,1.000000,0.0,0.0,0.0,0.0,0.406919,0.531687,0.560475,0.144969,0.578469,0.033879,0.788571,0.045584,0.751309,0.307258,0.364212,0.095278,0.049069,0.642077,0.642448,0.099334,0.118078,0.037744,0.883801,0.639676,0
230,0.642456,0.648345,0.275280,0.484472,0.390169,0.559406,0.924686,0.887097,0.764706,0.358696,0.409091,0.0,0.527778,0.316514,1.0,0.0,1.0,0.0,0.483146,0.75,0.411060,0.793767,0.434563,0.500000,0.0,0.0,0.0,0.0,0.438804,0.686702,0.665125,0.163326,0.739857,0.553035,0.910355,0.277936,0.865176,0.115375,0.116634,0.047338,0.046791,0.377204,0.521229,0.148066,0.088784,0.355801,0.753583,0.429150,1
781,0.408259,0.662385,0.832944,0.527950,0.396313,0.589109,0.991632,0.967742,0.588235,0.521739,0.409091,0.0,0.398148,0.610092,1.0,0.0,1.0,0.0,0.898876,0.00,0.505069,0.429664,0.155498,1.000000,0.0,0.0,0.0,0.0,0.400787,0.630086,0.620568,0.225170,0.753089,0.177080,0.608898,0.141267,0.757135,0.544081,0.469369,0.055984,0.128888,0.454864,0.504373,0.128855,0.152111,0.014481,0.900935,0.651822,0
353,0.547119,0.713739,0.388707,0.639752,0.327189,0.680693,0.845188,0.983871,0.470588,0.217391,0.136364,0.0,0.754630,0.311927,1.0,0.0,1.0,0.0,0.483146,0.75,0.349770,0.738738,0.427347,0.500000,0.0,0.0,0.0,0.0,0.516196,0.757354,0.759051,0.344803,0.900200,0.560684,0.973410,0.370320,1.000000,0.079149,0.079340,0.097206,0.030551,0.434241,0.397716,0.110283,0.060973,0.459816,0.734579,0.497976,1


,x,y,z,blavg,blmax,blmin,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,q6q6,angParam,entroAvgParam,chi2,chi3,chi4,chi7,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disord6,disordAvg6,disord8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Izz,degenDeg,surf
598,0.287043,0.243603,0.515268,0.651558,0.457308,0.678049,0.975369,0.918033,0.500000,0.298077,0.25,0.0,0.600,0.538813,1.0,0.0,1.0,0.0,0.976744,0.00,0.452921,0.857143,0.540842,0.034696,0.0,0.0,0.0,0.000000,0.345221,0.734086,0.588029,0.208956,0.676177,0.103868,0.653029,0.882287,0.195722,0.825850,0.491570,0.617988,0.447600,0.436040,0.208962,0.218740,0.179832,0.414844,0.576821,0.405594,0.499828,0.019585,0.720879,0.664122,1.0,0
432,0.457900,0.232383,0.423584,0.589235,0.616498,0.700000,0.881773,0.967213,0.611111,0.250000,0.35,0.0,0.720,0.296804,1.0,0.0,1.0,0.0,0.453488,0.75,0.388165,0.642857,0.648546,0.461708,0.0,0.0,0.0,0.000000,0.412086,0.850458,0.777565,0.292254,0.920466,0.595435,0.744681,0.938796,0.341748,0.939182,0.188345,0.170649,0.360228,0.157268,0.177667,0.176228,0.243240,0.247146,0.155384,0.374870,0.476425,0.372396,0.533242,0.461832,1.0,1
256,0.480320,0.280584,0.314144,0.711048,0.578871,0.631707,0.807882,0.983607,0.555556,0.182692,0.25,0.0,0.676,0.378995,1.0,0.0,1.0,0.0,0.453488,0.75,0.398214,0.642857,0.793777,0.463544,0.0,0.0,0.0,0.000000,0.472135,0.827931,0.731472,0.360550,0.857168,0.572829,0.766190,0.932782,0.359159,0.931429,0.265034,0.249015,0.303065,0.126704,0.187913,0.187733,0.209837,0.360596,0.281741,0.420193,0.509009,0.399569,0.567308,0.591603,1.0,1
789,0.252839,0.204840,0.563262,0.699717,0.560058,0.587805,0.965517,0.950820,0.388889,0.442308,0.45,0.1,0.476,0.666667,1.0,0.0,1.0,0.0,0.883721,0.00,0.535541,0.857143,0.543843,0.213669,0.0,0.0,0.0,0.333333,0.322246,0.698836,0.534570,0.166339,0.584838,0.179885,0.648663,0.842590,0.195714,0.711950,0.429688,0.604750,0.480327,0.415652,0.152997,0.208134,0.125813,0.353831,0.684476,0.248482,0.433144,0.039872,0.198626,0.343511,1.0,0
112,0.567459,0.504372,0.217611,0.577904,0.492041,0.358537,0.886700,0.918033,0.611111,0.269231,0.25,0.0,0.672,0.493151,1.0,0.0,1.0,0.0,0.488372,0.75,0.398958,0.642857,0.781601,0.397566,0.0,0.0,0.0,0.000000,0.398514,0.775697,0.743025,0.300574,0.723094,0.521187,0.641922,0.859124,0.295882,0.828055,0.952601,0.887711,0.474509,0.482816,0.237853,0.329970,0.348687,0.554665,0.518331,0.340875,0.592175,0.239162,0.634341,0.255725,1.0,1


,x,y,z,blavg,blmax,blmin,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,q6q6,angParam,entroParam,entroAvgParam,chi1,chi2,chi3,chi4,chi6,chi7,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disordAvg6,disordAvg8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Izz,degenDeg,surf
230,0.449644,0.436428,0.345310,0.661578,0.841010,0.652778,0.977778,0.941176,0.588235,0.471264,0.608696,0.000000,0.496894,0.537975,1.0,0.0,1.0,0.00,0.776596,0.000000,0.211176,0.857143,0.599451,0.318243,0.073731,0.714286,0.000000,0.000000,0.00,0.333333,0.00,0.329105,0.428637,0.750792,0.503545,0.552303,0.497752,0.107026,0.542793,0.797356,0.418687,0.560984,0.669858,0.469424,0.357151,0.278722,0.234096,0.314656,0.795162,0.919751,0.550991,0.523165,0.037971,0.605969,0.640678,1.0,0
273,0.839493,0.317842,0.377365,0.513995,0.635958,0.611111,0.960000,0.970588,0.411765,0.804598,0.739130,0.083333,0.130435,0.860759,0.5,0.0,1.0,0.25,0.723404,0.000000,0.654572,0.785714,0.367139,0.197679,0.438648,0.428571,0.666667,0.222222,0.50,0.600000,0.25,0.157044,0.172278,0.529833,0.390093,0.324489,0.556650,0.265669,0.187379,0.665967,0.169540,0.565208,0.258134,0.416057,0.550283,0.633082,0.754765,0.829095,0.493747,0.277335,0.790320,0.781734,0.156994,0.609803,0.501695,1.0,0
67,0.205144,0.520282,0.200401,0.707379,0.866270,0.384259,0.933333,0.985294,0.294118,0.505747,0.391304,0.166667,0.360248,0.816456,1.0,0.0,1.0,0.00,0.744681,0.000000,0.640058,0.928571,0.648244,0.214854,0.540081,0.714286,0.333333,0.111111,0.25,0.733333,0.25,0.251281,0.296637,0.578582,0.387990,0.285825,0.445155,0.417270,0.739795,0.835401,0.397927,0.516073,0.205138,0.222285,0.139658,0.129428,0.047722,0.111721,0.442517,0.146213,0.195945,0.169324,0.201325,0.674973,0.508475,1.0,0
400,0.946684,0.490003,0.541197,0.575064,0.638930,0.555556,0.697778,0.911765,0.411765,0.298851,0.434783,0.250000,0.360248,0.462025,1.0,0.0,1.0,0.00,0.393617,0.857143,0.735849,0.571429,0.352599,0.568410,0.640727,0.285714,0.333333,0.000000,0.00,0.333333,0.25,0.446911,0.439169,0.677061,0.526052,0.346403,0.532327,0.771107,0.653889,0.822511,0.204510,0.618007,0.279181,0.452230,0.752397,0.800247,0.666161,0.825135,0.576236,0.559209,0.642027,0.753248,0.426117,0.228368,0.481356,1.0,1
39,0.622670,0.537015,0.010077,0.704835,0.794948,0.430556,0.760000,0.985294,0.294118,0.310345,0.608696,0.166667,0.422360,0.500000,1.0,0.0,1.0,0.00,0.351064,0.857143,0.736575,0.571429,0.409027,0.602546,0.688705,0.285714,0.000000,0.111111,0.00,0.333333,0.00,0.578263,0.339096,0.694831,0.645755,0.435894,0.522786,0.683582,0.594831,0.686363,0.268814,0.380289,0.285749,0.210510,0.193666,0.236728,0.256172,0.496340,0.529503,0.484842,0.699973,0.521200,0.426432,0.744524,0.305085,1.0,1


,x,y,z,blavg,blmax,blmin,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,q6q6,angParam,entroParam,entroAvgParam,chi1,chi2,chi3,chi4,chi6,chi7,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q8avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disordAvg6,disord8,disordAvg8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Izz,degenDeg,surf
78,0.489350,0.331262,0.174661,0.541420,0.818182,0.642132,0.885593,0.952381,0.368421,0.666667,0.454545,0.000000,0.236686,0.764706,1.0,0.0,1.0,0.0,0.663265,0.000000,0.618861,0.846154,0.556780,0.431332,0.371307,0.428571,0.0,0.222222,0.0,0.400000,0.000000,0.202349,0.419689,0.577939,0.295671,0.447898,0.287055,0.210439,0.596062,0.802770,0.636736,0.410939,0.545412,0.650720,0.416811,0.513900,0.686435,0.230648,0.393292,0.306949,0.480516,0.494567,0.682443,0.729761,0.151526,0.546170,0.647510,1.0,0
752,0.634624,0.358666,0.757708,0.529586,0.671329,0.345178,0.894068,0.825397,0.315789,0.787879,0.681818,0.142857,0.171598,0.735294,1.0,0.0,1.0,0.0,0.704082,0.000000,0.543549,0.692308,0.814977,0.250873,0.384351,0.571429,0.0,0.222222,0.0,0.600000,0.333333,0.226876,0.105539,0.483633,0.505356,0.355311,0.125499,0.185363,0.368271,0.714880,0.613616,0.580312,0.284529,0.640183,0.691192,0.592960,0.570703,0.642355,0.790357,0.807793,0.640229,0.616898,0.526315,0.644538,0.226624,0.688615,0.716475,1.0,0
597,0.198377,0.514016,0.918080,0.162722,0.082168,0.624365,0.576271,0.507937,0.631579,0.555556,0.500000,0.142857,0.627219,0.488235,1.0,0.0,1.0,0.0,0.193878,0.714286,0.921415,0.461538,0.732060,0.766415,0.675672,0.000000,0.0,0.111111,0.2,0.200000,0.000000,0.577450,0.233436,0.607425,0.600554,0.586825,0.711410,0.485767,0.321371,0.703502,0.612003,0.399257,0.519090,0.290978,0.670759,0.342828,0.331121,0.777231,0.822055,0.948478,0.578858,0.698989,0.600781,0.629902,0.568635,0.260493,0.203065,1.0,1
442,0.293819,0.252330,0.813027,0.639053,0.818182,0.319797,0.444915,0.904762,0.368421,0.272727,0.272727,0.000000,0.591716,0.382353,1.0,0.0,1.0,0.0,0.234694,0.714286,0.785200,0.538462,0.673251,0.728948,0.605492,0.142857,0.0,0.000000,0.0,0.400000,0.000000,0.369789,0.285142,0.624563,0.922386,0.277809,0.617838,0.520804,0.281146,0.775906,0.733685,0.669702,0.397069,0.496405,0.684761,0.584027,0.585895,0.346603,0.748600,0.627035,0.637036,0.721480,0.440750,0.449694,0.695583,0.444911,0.302682,1.0,1
478,0.176226,0.677361,0.736240,0.494083,0.606643,0.677665,0.927966,0.968254,0.578947,0.575758,0.227273,0.142857,0.177515,0.829412,1.0,0.0,1.0,0.0,0.724490,0.000000,0.781270,0.923077,0.384021,0.229047,0.265033,0.857143,0.0,0.000000,0.2,0.666667,0.666667,0.196751,0.298489,0.780416,0.616986,0.198379,0.748024,0.291182,0.667876,0.842044,0.672623,0.143345,0.631589,0.409490,0.602737,0.285557,0.298426,0.866925,0.758468,0.756882,0.461951,0.629481,0.555866,0.651198,0.036251,0.713012,0.927203,1.0,0


,x,y,z,blavg,blmax,blmin,ba1avg,ba1max,ba1min,ba2avg,ba2max,ba2min,btposavg,btnegavg,btnegmax,btnegmin,btposmax,btposmin,gcn,scn,rad,q6q6,angParam,entroParam,entroAvgParam,chi1,chi2,chi3,chi4,chi5,chi6,chi7,q2,q4,q6,q8,q10,q12,q2avg,q4avg,q6avg,q10avg,q12avg,disord2,disordAvg2,disord4,disordAvg4,disordAvg6,disordAvg8,disord10,disordAvg10,disord12,disordAvg12,centParam,Ixx,Izz,degenDeg,surf
315,0.229420,0.804925,0.239332,0.565749,0.499121,0.508163,0.722944,0.767857,0.3125,0.571429,0.807692,0.1,0.68750,0.627219,1.00,0.0,1.0,0.0,0.219780,1.0,0.861357,0.285714,0.417381,0.685128,0.689780,0.166667,0.0,0.2,0.25,0.172414,0.1875,0.50,0.668485,0.211287,0.535325,0.565583,0.537293,0.572070,0.598459,0.435871,0.540066,0.500957,0.419410,0.991371,0.939156,0.355919,0.267015,0.475331,0.591599,0.369366,0.546143,0.476762,0.617547,0.364516,0.318511,0.060606,1.0,1
980,0.375703,0.613603,0.918008,0.458716,0.588752,0.436735,0.822511,0.857143,0.4375,0.554622,0.461538,0.2,0.44375,0.609467,1.00,0.0,1.0,0.0,0.318681,1.0,0.817109,0.500000,0.428006,0.569688,0.590067,0.500000,0.0,0.0,0.00,0.344828,0.3750,0.00,0.521598,0.354790,0.722001,0.626951,0.269794,0.596460,0.683164,0.804666,0.695479,0.399476,0.513107,0.397934,0.388115,0.778196,0.760479,0.575868,0.547795,0.758144,0.618318,0.802071,0.830357,0.370062,0.590486,0.477273,1.0,1
16,0.123590,0.522046,0.265393,0.590214,0.968366,0.512245,0.714286,0.857143,0.1875,0.470588,0.923077,0.2,0.61875,0.579882,0.75,0.0,1.0,0.5,0.351648,1.0,0.789971,0.642857,0.468184,0.683227,0.573373,0.333333,0.0,0.0,0.00,0.241379,0.2500,0.00,0.441353,0.460102,0.739043,0.631561,0.517446,0.504751,0.928332,0.819347,0.848905,0.755728,0.540914,0.628665,0.496729,0.018842,0.089268,0.511607,0.677322,0.340842,0.299622,0.364831,0.438005,0.454873,0.267322,0.204545,1.0,1
212,0.694892,0.459884,0.149384,0.541284,0.959578,0.532653,0.943723,0.946429,0.4375,0.731092,0.384615,0.1,0.18750,0.798817,1.00,0.0,1.0,0.0,0.648352,0.0,0.723894,0.428571,0.204791,0.337447,0.503379,0.500000,0.0,0.4,0.00,0.517241,0.6250,0.25,0.263101,0.235348,0.457869,0.473564,0.287311,0.519066,0.433561,0.414715,0.523292,0.315825,0.407023,0.324556,0.493164,0.622834,0.443145,0.402407,0.476883,0.501076,0.526864,0.581748,0.628682,0.185537,0.635729,0.484848,1.0,0
66,0.572906,0.507349,0.043648,0.577982,0.964851,0.328571,0.675325,0.946429,0.1250,0.521008,0.807692,0.0,0.59375,0.532544,1.00,0.0,1.0,0.0,0.395604,1.0,0.843068,0.714286,0.446069,0.553216,0.593867,0.500000,0.0,0.0,0.00,0.344828,0.3125,0.00,0.591641,0.286708,0.800288,0.614669,0.418497,0.577494,0.817267,0.672482,0.777578,0.706588,0.463633,0.546019,0.484208,0.512887,0.679834,0.176280,0.270674,0.426184,0.503596,0.313380,0.572623,0.396877,0.605739,0.420455,1.0,1
